# SIADS 516: Homework 2

- **Dr. Chris Teplovs**, School of Information, University of Michigan
- **Kris Steinhoff**, School of Information, University of Michigan


In [1]:
# The AutograderHelper class provides methods used by the autograder.
from autograder_helper import AutograderHelper

In [2]:
# Autograder cell. This cell is worth 0 points.
# This cell has hidden code used to configure the autograder.

# Using the Spark RDD API to analyze text
Data are from 
https://www.kaggle.com/nzalake52/new-york-times-articles

## Objectives
1. To gain familiarity with PySpark
2. To learn the basics of the Spark RDD API
3. To practice solving a real-world problem

## Overview

This project was inspired by an actual event that was experienced by a UMSI student.  This student was applying for a 
job with a large multi-national corporation (let's call it XYZ, Inc.).  XYZ Inc. was looking for someone who could 
conduct an analysis of a massive (terabyte-size) text dataset.  They had heard about Spark and planned on investigating it but hadn't yet found someone internally who had the skill set required to tackle the problem.  The UMSI student indicated that they had experience with Spark and could likely handle the task.  The hiring supervisor then provided a non-Spark script and asked the student to demonstrate how that script could be translated to work in a Spark environment.  The student was able to do the conversion and, pending completion of their degree, will have secured a job at XYZ, Inc.

This assignment simulates that exact situation.  **In this assignment you will take a python-based script that does
part-of-speech tagging on a large dataset and convert it, as much as possible, to use a pyspark-based approach.**

---

### Task: Review non-Spark code

The original script was written by Luke Petschauer and a forked version is available in this notebook: [NP_chunking_with_the_NLTK.ipynb](https://github.com/umsi-data-science/NP_chunking_with_nltk/blob/master/NP_chunking_with_the_NLTK.ipynb).

It provides a detailed explanation of the original code and an excellent overview and justification for the use of
part-of-speech tagging and a super-gentle introduction to Natural Language Processing (NLP).

Let's use some of the code from that notebook here...

We'll start by importing the required packages, and making sure the NLTK collections are downloaded to your environment.

In [3]:
import nltk
import re
import pprint
from nltk import Tree

nltk.download('book') # NOTE: this should be unnecessary for Coursera image (should be preloaded)

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/jovyan/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_t

True

The code in the next cell is from the "Final Code" section in the NP_chunking_with_the_NLTK.ipynb notebook (linked above). This implementation doesn't use Spark. 

In [4]:
# This is the original (non-Spark) script

patterns = """
    NP: {<JJ>*<NN*>+}
    {<JJ>*<NN*><CC>*<NN*>+}
    """

NPChunker = nltk.RegexpParser(patterns)

def prepare_text(input):
    sentences = nltk.sent_tokenize(input)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPChunker.parse(sent) for sent in sentences]
    return sentences


def parsed_text_to_NP(sentences):
    nps = []
    for sent in sentences:
        tree = NPChunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps


def sent_parse(input):
    sentences = prepare_text(str(input))
    nps = parsed_text_to_NP(sentences)
    return nps

In [5]:
text_to_be_analyzed = """\
WASHINGTON - Stellar pitching kept the Mets afloat in the first half of last season despite their offensive 
woes. But they cannot produce an encore of their pennant-winning season if their lineup keeps floundering 
while their pitching is nicked, bruised and stretched thin.

"We were going to ride our pitching," Manager Terry Collins said before Wednesday’s game. "But we're not 
riding it right now. We've got as many problems with our pitching as we do anything."

Wednesday's 4-2 loss to the Washington Nationals was cruel for the already-limping Mets. Pitching in Steven 
Matz's place, the spot starter Logan Verrett allowed two runs over five innings. But even that was too large 
a deficit for the Mets' lineup to overcome against Max Scherzer, the Nationals' starter.

"We're not even giving ourselves chances," Collins said, adding later, "We just can’t give our pitchers any 
room to work."

The Mets did not score until the ninth inning, when a last-gasp two-run homer by James Loney off Nationals 
reliever Shawn Kelley snapped a streak of 23 scoreless innings for the team.
"""


nps = sent_parse(text_to_be_analyzed)

# Print a list of noun phrases found in text_to_be_analyzed
print(nps)

['Stellar pitching', 'afloat', 'first half', 'last season', 'encore', 'pennant-winning season', 'lineup', 'pitching', 'thin', 'pitching', 's game', 'pitching', 'anything', '4-2 loss', 'place', 'spot starter', 'deficit', 'lineup', 'starter', 'room', 'ninth inning', 'last-gasp two-run homer', 'reliever', 'streak', 'team']


You will be taking a similar approach to analyze a large set of news articles from the New York Times using pyspark.

**Before you continue to the next task, you should:**
- read through and study the NP_chunking_with_the_NLTK.ipynb notebook (linked above)
- study and run the cells above.

---

In [6]:
# Import packages and set up a Spark Session and Context.

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('SIADS 516 Homework 2') \
    .getOrCreate() 

sc = spark.sparkContext

22/07/08 07:02:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


---

## -- PART-OF-SPEECH COUNT --

### Task: Create an RDD pipline to show the count of each part-of-speach tag sorted in descending order

Complete the implementation of the `pos_counts()` function below so that it uses an RDD pipeline (i.e. sequence of transformations) to:
1. filter out blank lines 
2. filter out lines starting with 'URL'
3. create a single list (using flatMap) that applies the `pos_tag_counter()` (this is defined below for you below) function to each line
4. map each resulting line to show the part of speech (which is the second element returned from the pos_tag_counter)
5. convert each resulting line to a pairRDD with POS tags as keys and values of 1
6. reduce the resulting RDD by key, adding up all the 1s (like the lecture and lab examples)
7. sort the resulting list by the counts, in descending order

In [7]:
# This is the function you will use with flatMap in your pipeline.

TOKEN_RE = re.compile(r"\b[\w']+\b")
def pos_tag_counter(line):
    toks = nltk.regexp_tokenize(line, TOKEN_RE)
    postoks = nltk.tag.pos_tag(toks)
    return postoks

In [8]:
def pos_counts(rdd):
    

    rdd = rdd.filter(lambda x: len(x) > 0)
    rdd = rdd.filter(lambda x: str(x)[:3]!= "URL")
    pos_total_sorted = rdd.flatMap(lambda x: pos_tag_counter(x))
    pos_total_sorted = pos_total_sorted.map(lambda x: (x[-1],1))
    pos_total_sorted=pos_total_sorted.reduceByKey(lambda a,b: a+b)
    pos_total_sorted = pos_total_sorted.sortBy(lambda x: -x[1])
    

    
    return pos_total_sorted  # This should be the final stage of your pipeline, an RDD with the 
                             # count of each part-of-speach tag sorted in descending order.

Let's start by trying your code on a small data set. The `text_to_be_analyzed` from the cells above will do nicely. We can use the parallelize() method to turn it into a RDD, pass that to your function, and then take() the first ten entries:

In [9]:
small_text = sc.parallelize(text_to_be_analyzed.split("\n"))

small_pos_counts = pos_counts(small_text)

In [10]:
small_pos_counts_take_10 = small_pos_counts.take(10)
small_pos_counts_take_10

[('NN', 30),
 ('NNP', 24),
 ('IN', 20),
 ('DT', 16),
 ('VBD', 11),
 ('RB', 11),
 ('JJ', 11),
 ('NNS', 10),
 ('PRP$', 7),
 ('VB', 7)]

In [11]:
# Autograder cell. This cell is worth 2 points (out of 20). This cell does not contain hidden tests.
# This cell deliberately includes answers to provide guidance on how this question is graded.

correct = AutograderHelper.parse_spark_take([
    ('NN', 30),
    ('NNP', 24),
    ('IN', 20),
    ('DT', 16),
    ('VBD', 11),
    ('JJ', 11),
    ('RB', 11),
    ('NNS', 10),
    ('PRP$', 7),
    ('VB', 7),
])

AutograderHelper.assert_same_shape(
    correct=correct,
    submitted=AutograderHelper.parse_spark_take(small_pos_counts_take_10),
)

Now let's run it against a much larger data set. *The complete analysis could take about 10 minutes to run.*

In [12]:
text = sc.textFile('../../assets/data/nytimes/nytimes_news_articles.txt')

pos_counts = pos_counts(text)

In [13]:
pos_counts_take_10 = pos_counts.take(10)
pos_counts_take_10

[('NN', 1126515),
 ('IN', 928916),
 ('NNP', 853093),
 ('DT', 761492),
 ('JJ', 498482),
 ('NNS', 437116),
 ('VBD', 379509),
 ('PRP', 282603),
 ('RB', 271053),
 ('CC', 231491)]

In [14]:
# Autograder cell. This cell is worth 8 points (out of 20). This cell contains hidden tests.

---

## -- NOUN PHRASE LENGTH --

### Task: Create an RDD pipeline to show the distribution of the length of noun phrases

Complete the implementation of the `noun_phrase_length_distribution()` function below so that it uses an RDD pipeline  to return a PairRDD which contains the distribution of the length of noun phrases.

- You can apply the `tokenize_chunk_parse()` (defined below for you) to apply--with flatMap()--to each entry in the input RDD.
- Sorting the resulting list by the counts in descending order will make the results easier to interpret.

In [15]:
# This cell defines the tokenize_chunk_parse() function you will use with flatMap()

grammar = r"""
    NBAR:
        {<NN.*|JJS>*<NN.*>}
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}
"""

  
def tokenize_chunk_parse(line):
    chunker = nltk.RegexpParser(grammar)
  
    toks = nltk.regexp_tokenize(line, TOKEN_RE)
    postoks = nltk.tag.pos_tag(toks)

    if len(postoks) == 0:
        return []
    
    tree = chunker.parse(postoks)

    return [term for term in leaves(tree)] 
  
def leaves(tree):
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

In [16]:
def noun_phrase_length_distribution(rdd):

    # YOUR CODE HERE
    #raise NotImplementedError()
    distribution = rdd.flatMap(lambda x: tokenize_chunk_parse(x))
    #distribution = rdd.flatMap(lambda x: leaves(distribution))
    #distribution = distribution.map(lambda x: (x[-1],1))
    distribution = distribution.map(lambda x: (len(x),1))
    distribution=distribution.reduceByKey(lambda a,b: a+b)
    distribution = distribution.sortBy(lambda x: -1*x[1])

    return distribution  # This should be the final stage of your pipeline, a PairRDD with the 
                         # distribution of the length of noun phrases.

In [17]:
small_counts = noun_phrase_length_distribution(small_text)

In [18]:
small_counts_take_10 = small_counts.take(10)
small_counts_take_10

[(1, 29), (2, 10), (3, 3), (4, 2)]

The cell above should produce this output:

```
[(1, 29), (2, 10), (3, 3), (4, 2)]
```

This means there are 29 1-word noun phrases, 10 2-word noun phrases, 3 3-word noun phrases, and 2 4-word noun phrases in the `small_text` data set.

In [19]:
# Autograder cell. This cell is worth 2 points (out of 20). This cell does not contain hidden tests.
# This cell deliberately includes answers to provide guidance on how this question is graded.

correct = AutograderHelper.parse_spark_take(
    [(1, 29), (2, 10), (3, 3), (4, 2)]
)

AutograderHelper.assert_same_shape(
    correct=correct,
    submitted=AutograderHelper.parse_spark_take(small_counts_take_10),
)

Now let's run it against the larger data set. *The complete analysis could take about 10 minutes to run.*

In [20]:
text = sc.textFile('../../assets/data/nytimes/nytimes_news_articles.txt')

counts = noun_phrase_length_distribution(text)

In [21]:
counts_take_10 = counts.take(10)
counts_take_10

[(1, 1205976),
 (2, 353457),
 (3, 119065),
 (4, 35890),
 (5, 11079),
 (6, 3889),
 (7, 1400),
 (8, 543),
 (9, 257),
 (10, 112)]

In [22]:
# Autograder cell. This cell is worth 8 points (out of 20). This cell contains hidden tests.